In [44]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import ipywidgets as widgets
from IPython.display import display 
import datetime

In [45]:
def total_cases(resizer = 585, expected_days = 30,scenarios = 100, start_date = '2020-01-15', show_expectations = True, show_daily_death = False, show_daily_new_cases = False, show_total_death = False, remove_before_15feb=False):
    #read Actual Database
    covid_db = pd.read_csv("coviddata.csv")
    covid_db["day"] = pd.to_datetime(covid_db["date"],format = "%Y-%m-%d")
    covid_db.index = covid_db["day"]
    covid_egy = covid_db.query('iso_code == "EGY"')[start_date:]
    #covid_egy.replace({ 0 :None},inplace = True)
    if remove_before_15feb:
        start_date = '2020-02-15'
        covid_egy = covid_egy.query('total_cases != 0')
    #print(covid_egy.head(50))
    ax = covid_egy["total_cases"].plot(figsize = (30,15), alpha = 0.7, color = "blue", label = "Actual total Cases",legend = True,title = "COVID19 Cases in Egypt");
    if show_daily_new_cases: covid_egy["new_cases"].plot( color = "indianred", alpha = 0.7, label = "New Cases", legend = True, secondary_y = True);
    if show_daily_death: covid_egy["new_deaths"].plot(alpha = 0.5, color = "red", style = ":", label = "New Death",legend = True,secondary_y = True);
    if show_total_death: covid_egy["total_deaths"].plot(alpha = 0.7, color = "black", style = ":", label = "Total Death",legend = True,secondary_y = True);
    act_last_day = covid_egy["total_cases"][-1]
    
    n_of_days =  len(covid_egy.index)
    today_cases = 1
    cases_history = pd.DataFrame(index = covid_egy.index,columns = ('E',) )
    size = resizer / 10000
    
    lastday = cases_history.index[-1]
    next_date = lastday + datetime.timedelta(days=1)   
    next_days = pd.DataFrame(index = pd.date_range(start = next_date, periods = int(expected_days)), columns = np.arange(0,scenarios,1))

    #the current days as model
    for i in range(n_of_days):
        x = i * size
        today_cases = today_cases + np.exp(x)
        cases_history['E'].iloc[i] = today_cases
        
    #Plotting the current days model
    ax.plot(cases_history["E"],alpha = 0.7, ls = "--",label = "Expectation Model for past data",color = 'orange')
    
    cases_last_day = round(cases_history["E"][-1],0)

    if show_expectations:
        
        #find covariances between model and Actual Data 
        diff = covid_egy["total_cases"] - cases_history['E']
        diff_per = diff / covid_egy["total_cases"].replace(0,1)
        var_range = pd.DataFrame(index = covid_egy.index, columns = ('var',),data =diff_per)

        #expexting the next days
        for scenario in range(scenarios):
            expexted_cases = today_cases
            start_day = next_days.index[0]
            day_n = n_of_days +1
            day_index = 0  
            for day in next_days.index:
                #print(day)
                x = day_n * size
                #print(expexted_cases)
                expexted_cases = expexted_cases + np.exp(x) + float(np.random.normal(loc = var_range.mean(),scale = var_range.std()))
                #print(expexted_cases)
                #print(pd.to_datetime(day, format = "%Y-%m-%d"))
                next_days[scenario].iloc[day_index] = expexted_cases
                start_day +=datetime.timedelta(days=1)
                day_index +=1
                day_n +=1
            scenario +=1
        
        cases_next_day = next_days.mean(axis = 1)[0]        
        cases_last_e_day = next_days.mean(axis = 1)[-1]
        ax.plot(next_days,ls = ":",label = "Expectation Model for Next days",alpha = 0.3,color = 'orange')
        ax.axhline(y=cases_last_e_day, ls="--", color="red", linewidth=2)
        ax.annotate(f"Expexted Next day cases: {round(cases_next_day,0)} \nExpexted at the end of {expected_days} days: {round(cases_last_e_day,0)}".format(a =expected_days), xy=(.5, .7), xycoords='axes fraction', fontsize=15)
          
        
        #find mean of expectations
        next_days["mean_"]= next_days.mean(axis = 1)
        next_days["min_"] = next_days.min(axis = 1)
        next_days["max_"] = next_days.max(axis = 1)

        #plot mean, min, max
        ax.plot(next_days["mean_"], ls=":", color="black", linewidth=1,label = "Mean Model for Next days",alpha = 0.7)
        ax.plot(next_days["min_"], ls=":", color="yellow", linewidth=1,label = "Min Model for Next days",alpha = 0.7)
        ax.plot(next_days["max_"], ls=":", color="red", linewidth=1,label = "Max Model for Next days",alpha = 0.7)

        # model without adding random variances
        expexted_cases = today_cases
        start_day = next_days.index[0]
        day_n = n_of_days +1
        day_index = 0  
        old_model = []
        
        for day in next_days.index:
            x = day_n * size
            expexted_cases = expexted_cases + np.exp(x)
            old_model.append(expexted_cases)
            day_n +=1
        next_days["old_model"] = old_model
    
    
        max_next_day = round(next_days["max_"][0],0)
        min_next_day = round(next_days["min_"][0],0)
        mean_next_day = round(next_days["mean_"][0],0)
        exp_next_day = round(next_days["old_model"][0],0)

        best_case_new = min_next_day - act_last_day
        wrost_case_new = max_next_day - act_last_day
        average_new = mean_next_day - act_last_day
        exp_new = exp_next_day - act_last_day

        new_cases = {"New Cases Scnarios":"" ,
                     "Best Case":best_case_new,
                     "Wrost Case": wrost_case_new,
                     "Average New cases": average_new,
                     "Expected new cases": exp_new
                    }
    
        ax.annotate(f"New Cases Scnarios:\nBest Case: {best_case_new}\nWrost Case: {wrost_case_new}\nAverage New cases: {average_new}\nExpected new cases: {exp_new}", xy=(.3, .5), xycoords='axes fraction', fontsize=15)
        ax.annotate(f"Max next day : {max_next_day}\nMin next day cases: {cases_last_day,} \nMean next day:{mean_next_day}\nExpected next day:{exp_next_day}", xy=(.3, .7), xycoords='axes fraction', fontsize=15)

    ax.axhline(y=cases_last_day, ls="--", color="black", linewidth=2,alpha = 0.5)
    ax.annotate(f"Actual Last day cases : {act_last_day}\nExpexcted Last day cases: {cases_last_day}", xy=(.1, .7), xycoords='axes fraction', fontsize=15)


    

In [46]:

widgets.interactive(total_cases, resizer = widgets.IntSlider(value = 535, min = 400, max = 800, step = 10 , description="Exponensial Percentage"),
                   expected_days = widgets.IntSlider(value = 5, min = 0, max = 100, description = "Expected days"),
                   scenarios = widgets.IntSlider(value = 50, min = 0, max = 100, description = "Scenarios"),
                   start_date = widgets.DatePicker(value =datetime.date(2020,1,15)),
                   show_expectations = True,
                   remove_before_15feb = False)

interactive(children=(IntSlider(value=535, description='Exponensial Percentage', max=800, min=400, step=10), I…